In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install datasets==1.0.2
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir '/content/gdrive/My Drive/t5'
!mkdir '/content/gdrive/My Drive/t5/dataset'
!mkdir '/content/gdrive/My Drive/t5/model'
!mkdir '/content/gdrive/My Drive/t5/tokenizer'

mkdir: cannot create directory ‘/content/gdrive/My Drive/t5’: File exists
mkdir: cannot create directory ‘/content/gdrive/My Drive/t5/dataset’: File exists
mkdir: cannot create directory ‘/content/gdrive/My Drive/t5/model’: File exists
mkdir: cannot create directory ‘/content/gdrive/My Drive/t5/tokenizer’: File exists


In [ ]:
#load the dataset
from datasets import load_dataset
train_data=load_dataset('squad',split='train')
test_data=load_dataset('squad',split='validation')

Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/1244d044b266a5e4dbd4174d23cb995eead372fbca31a03edc3f8a132787af41)
Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/1244d044b266a5e4dbd4174d23cb995eead372fbca31a03edc3f8a132787af41)


In [ ]:
type(train_data)

datasets.arrow_dataset.Dataset

In [ ]:
train_data

Dataset(features: {'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}, num_rows: 87599)

In [ ]:
from pprint import pprint 
sample_validation_dataset = next(iter(test_data))
pprint (sample_validation_dataset)

{'answers': {'answer_start': [177, 177, 177],
             'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']},
 'context': 'Super Bowl 50 was an American football game to determine the '
            'champion of the National Football League (NFL) for the 2015 '
            'season. The American Football Conference (AFC) champion Denver '
            'Broncos defeated the National Football Conference (NFC) champion '
            'Carolina Panthers 24–10 to earn their third Super Bowl title. The '
            "game was played on February 7, 2016, at Levi's Stadium in the San "
            'Francisco Bay Area at Santa Clara, California. As this was the '
            '50th Super Bowl, the league emphasized the "golden anniversary" '
            'with various gold-themed initiatives, as well as temporarily '
            'suspending the tradition of naming each Super Bowl game with '
            'Roman numerals (under which the game would have been known as '
            '"Super

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm


In [ ]:
df_train=pd.DataFrame(columns=['context','question'])
df_test=pd.DataFrame(columns=['context','question'])

In [ ]:
count=0
for _,val in enumerate(tqdm(train_data)):
  passage=val['context']
  question=val['question']
  df_train.loc[count]=[passage] + [question]
  count=count+1


  0%|          | 0/87599 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(lst) 

,context,question
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...


In [ ]:
count=0
for _,val in enumerate(tqdm(test_data)):
  passage=val['context']
  question=val['question']
  df_test.loc[count]=[passage] + [question]
  count=count+1


  0%|          | 0/10570 [00:00<?, ?it/s]

In [ ]:
df_test.head()

,context,question
0,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...
1,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...
2,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?
3,Super Bowl 50 was an American football game to...,Which NFL team won Super Bowl 50?
4,Super Bowl 50 was an American football game to...,What color was used to emphasize the 50th anni...


In [ ]:
from sklearn.utils import shuffle
df_train = shuffle(df_train)
df_test = shuffle(df_test)

In [ ]:
train_save_path = '/content/gdrive/My Drive/t5/dataset/squad_t5_train.csv'
test_save_path = '/content/gdrive/My Drive/t5/dataset/squad_t5_test.csv'
#df_train.to_csv(train_save_path, index = False)
#df_test.to_csv(test_save_path, index = False)

#Train T5 Transformer Model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install --quiet transformers
!pip install --quiet tokenizers
!pip install --quiet sentencepiece
!pip install --quiet tqdm
!pip install --quiet pytorch-lightning
!pip install --quiet torchtext


In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from termcolor import colored
import textwrap

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

pl.seed_everything(42)

Global seed set to 42


42

In [ ]:
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:169: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


#Train Dataset Preparation

In [ ]:
from tqdm.notebook import tqdm
import copy

class QuestionGenerationDataset(Dataset):
    def __init__(self, tokenizer, filepath, max_len_input=512,max_len_out=96):
        self.path = filepath
        self.passage_column = "context"
        #self.answer = "answer"
        self.question = "question"

        #self.data = pd.read_csv(self.path)
        
        self.data = pd.read_csv(self.path,nrows=1000)
        
        self.data=pd.read_csv(self.path,nrows=250)  

        self.max_len_input = max_len_input
        self.max_len_output = max_len_out
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self.skippedcount =0
        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        labels = copy.deepcopy(target_ids)
        labels [labels==0] = -100

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask,"labels":labels}

    def _build(self):
        for idx in tqdm(range(len(self.data))):
            passage,target = self.data.loc[idx, self.passage_column],self.data.loc[idx, self.question]

            input_ = "context: %s  </s>" % (passage)
            target = "question: %s </s>" % (str(target))

            # get encoding length of input. If it is greater than self.max_len skip it
            test_input_encoding = self.tokenizer.encode_plus(input_,
                                        truncation=False,
                                        return_tensors="pt")
            
            length_of_input_encoding = len(test_input_encoding['input_ids'][0])


            if length_of_input_encoding > self.max_len_input:
              self.skippedcount = self.skippedcount + 1
              continue

            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], max_length=self.max_len_input, pad_to_max_length=True, return_tensors="pt"
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length=self.max_len_output, pad_to_max_length=True,return_tensors="pt"
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)


In [ ]:
train_dataset = QuestionGenerationDataset(t5_tokenizer,train_save_path)

  0%|          | 0/250 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:215: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padd

In [ ]:
train_sample=train_dataset[0]
decoded_train_input=t5_tokenizer.decode(train_sample['source_ids'])
decoded_train_output=t5_tokenizer.decode(train_sample['target_ids'])

In [ ]:
print(decoded_train_input)
print(decoded_train_output)

context: Few large stone buildings were constructed between the Constantinian basilicas of the 4th century and the 8th century, although many smaller ones were built during the 6th and 7th centuries. By the beginning of the 8th century, the Carolingian Empire revived the basilica form of architecture. One feature of the basilica is the use of a transept, or the "arms" of a cross-shaped building that are perpendicular to the long nave. Other new features of religious architecture include the crossing tower and a monumental entrance to the church, usually at the west end of the building.</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <

In [ ]:
test_dataset = QuestionGenerationDataset(t5_tokenizer,test_save_path)

  0%|          | 0/250 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:215: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


#Fine Tune Model

In [ ]:
class T5FineTuner(pl.LightningModule):
    def __init__(self,hparams, t5model, t5tokenizer):
        super(T5FineTuner, self).__init__()
        self.save_hyperparameters(hparams)
        self.model = t5model
        self.tokenizer = t5tokenizer


    def forward( self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None):
         outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=decoder_attention_mask,
            labels=lm_labels,
        )
         
         return outputs


    def training_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_input_ids = batch["target_ids"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log('train_loss',loss)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_input_ids = batch["target_ids"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log("val_loss",loss)
        return loss

    def train_dataloader(self):
        return DataLoader(train_dataset, batch_size=self.hparams.batch_size,num_workers=4)

    def val_dataloader(self):
        return DataLoader(test_dataset, batch_size=self.hparams.batch_size,num_workers=4)



    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=3e-4, eps=1e-8)
        return optimizer



In [ ]:
args_dict = dict(
    batch_size=4,
)

args = argparse.Namespace(**args_dict)


model = T5FineTuner(args,t5_model,t5_tokenizer)

trainer = pl.Trainer(max_epochs = 1, gpus=1,progress_bar_refresh_rate=30)

trainer.fit(model)

print ("Saving model")
save_path_model = '/content/gdrive/My Drive/t5/model/'
save_path_tokenizer = '/content/gdrive/My Drive/t5/tokenizer/'
model.model.save_pretrained(save_path_model)
t5_tokenizer.save_pretrained(save_path_tokenizer)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=30)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/tr

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (_ResultMetric). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchme

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Saving model


('/content/gdrive/My Drive/t5/tokenizer/tokenizer_config.json',
 '/content/gdrive/My Drive/t5/tokenizer/special_tokens_map.json',
 '/content/gdrive/My Drive/t5/tokenizer/spiece.model',
 '/content/gdrive/My Drive/t5/tokenizer/added_tokens.json')

In [ ]:
!pip install git+https://github.com/Maluuba/nlg-eval.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Maluuba/nlg-eval.git (to revision master) to /tmp/pip-req-build-mzcu8k70
  Running command git clone -q https://github.com/Maluuba/nlg-eval.git /tmp/pip-req-build-mzcu8k70
     |████████████████████████████████| 1.5 MB 5.1 MB/s 
     |████████████████████████████████| 281 kB 73.9 MB/s 
     |████████████████████████████████| 24.2 MB 97.1 MB/s 
     |████████████████████████████████| 2.8 MB 55.5 MB/s 
     |████████████████████████████████| 749 kB 61.3 MB/s 
  Created wheel for nlg-eval: filename=nlg_eval-2.3-py3-none-any.whl size=68175165 sha256=20ea6762228c214961f1a49480f74fd7c852ead27c2aa47dca32e1de529eb0c4
  Stored in directory: /tmp/pip-ephem-wheel-cache-m20b33j1/wheels/08/20/df/33ced66932f198c4323042d18ff1c2db9b9716369f0de4afb4
  Created wheel for Theano: filename=Theano-1.0.5-py3-none-any.whl size=2668111 sha256=6e81f8492bd14d70dae0056a8671e09f09933fccfe

In [ ]:
!nlg-eval --setup


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Installing to /root/.cache/nlgeval
In case of incomplete downloads, delete the directory and run `nlg-eval --setup /root/.cache/nlgeval' again.
glove2word2vec.py: 100% 1.00/1.00 [00:00<00:00, 379 chunks/s]
dictionary.txt: 550 chunks [00:00, 880 chunks/s]
uni_skip.npz: 100% 634/634 [00:25<00:00, 24.4 chunks/s]
uni_skip.npz.pkl: 100% 1.00/1.00 [00:00<00:00, 1.37k chunks/s]
bi_skip.npz: 100% 276/276 [00:12<00:00, 22.9 chunks/s]
bi_skip.npz.pkl: 100% 1.00/1.00 [00:00<00:00, 2.07k chunks/s]
multi-bleu.perl: 100% 1.00/1.00 [00:00<00:00, 457 chunks/s]
utable.npy: 100% 2.23k/2.23k [00:59<00:00, 37.6 chunks/s]
btable.npy: 100% 2.23k/2.23k [01:01<00:00, 36.1 chunks/s]
glove.6B.zip: 100% 823/823 [02:42<00:00, 5.06 chunks/s]
2022-06-06 11:32:58,493 : MainThread : INFO : 400000 lines with 300 dimensions
2022-06-06 11:33:02,512 : MainThread : INFO : Model /root/.cache/nlgeval/glove.6B.300d.model

In [ ]:
!pip install pytest



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Test the model

In [ ]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer

In [ ]:
model= '/content/gdrive/My Drive/t5/model/'
tokenizer = '/content/gdrive/My Drive/t5/tokenizer/'

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(model)
tokenizer = T5Tokenizer.from_pretrained(tokenizer)

In [ ]:
data=pd.read_csv('/content/gdrive/My Drive/t5/dataset/squad_t5_test.csv')

In [ ]:
data.head(5)

,context,question
0,His poor physical health made him short-temper...,How many times did Luther preach in Halle in 1...
1,Luther refused to recant his writings. He is s...,"Who said, ""Here I stand. I can do no other""?"
2,The Premier of Victoria is the leader of the p...,Who sets the legislative agenda in Victoria?
3,"In 2006, a toxic waste spill off the coast of ...",In which year did a toxic waste spill from a E...
4,Although Kenya is the most industrially develo...,What are the three largest urban areas in Kenya?


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

device  cuda


In [ ]:
lis_context=[]
for ind in data.index:
  #print(data['context'][ind])
  context='context:'+data['context'][ind] + ' </s>'
  lis_context.append(context)

In [ ]:
for item in lis_context:
  print(item)

context:His poor physical health made him short-tempered and even harsher in his writings and comments. His wife Katharina was overheard saying, "Dear husband, you are too rude," and he responded, "They are teaching me to be rude." In 1545 and 1546 Luther preached three times in the Market Church in Halle, staying with his friend Justus Jonas during Christmas. </s>
context:Luther refused to recant his writings. He is sometimes also quoted as saying: "Here I stand. I can do no other". Recent scholars consider the evidence for these words to be unreliable, since they were inserted before "May God help me" only in later versions of the speech and not recorded in witness accounts of the proceedings. However, Mullett suggests that given his nature, "we are free to believe that Luther would tend to select the more dramatic form of words." </s>
context:The Premier of Victoria is the leader of the political party or coalition with the most seats in the Legislative Assembly. The Premier is the 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



context:Private schooling in the United States has been debated by educators, lawmakers and parents, since the beginnings of compulsory education in Massachusetts in 1852. The Supreme Court precedent appears to favor educational choice, so long as states may set standards for educational accomplishment. Some of the most relevant Supreme Court case law on this is as follows: Runyon v. McCrary, 427 U.S. 160 (1976); Wisconsin v. Yoder, 406 U.S. 205 (1972); Pierce v. Society of Sisters, 268 U.S. 510 (1925); Meyer v. Nebraska, 262 U.S. 390 (1923). </s>
context:Luther secretly returned to Wittenberg on 6 March 1522. He wrote to the Elector: "During my absence, Satan has entered my sheepfold, and committed ravages which I cannot repair by writing, but only by my personal presence and living word." For eight days in Lent, beginning on Invocavit Sunday, 9 March, Luther preached eight sermons, which became known as the "Invocavit Sermons". In these sermons, he hammered home the primacy of core C

In [ ]:
model.eval()
lis_questions=[]
count=0
for text in lis_context:
  encoding = tokenizer.encode_plus(text,max_length =512, padding=True, return_tensors="pt")
  input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
  beam_outputs = model.generate(
    input_ids=input_ids,attention_mask=attention_mask,
    max_length=72,
    early_stopping=True,
    num_beams=8,
    num_return_sequences=8)
  lis_q=[]
  for beam_output in beam_outputs:
    #lis_q=[]
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    #print("Sent:")
    #print(sent)
    wor='question:'
    words=sent.split()
    for word in words:
        if wor==word:
          words.remove(wor)
          break
    sent=' '.join(words)
    lis_q.append(sent)
  lis_questions.append(lis_q)
  count=count+1
  print("Count")
  print(count)
  if count==300:
    break





/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2302: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:215: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


Count
1
Count
2
Count
3
Count
4
Count
5
Count
6
Count
7
Count
8
Count
9
Count
10
Count
11
Count
12
Count
13
Count
14
Count
15
Count
16
Count
17
Count
18
Count
19
Count
20
Count
21
Count
22
Count
23
Count
24
Count
25
Count
26
Count
27
Count
28
Count
29
Count
30
Count
31
Count
32
Count
33
Count
34
Count
35
Count
36
Count
37
Count
38
Count
39
Count
40
Count
41
Count
42
Count
43
Count
44
Count
45
Count
46
Count
47
Count
48
Count
49
Count
50
Count
51
Count
52
Count
53
Count
54
Count
55
Count
56
Count
57
Count
58
Count
59
Count
60
Count
61
Count
62
Count
63
Count
64
Count
65
Count
66
Count
67
Count
68
Count
69
Count
70
Count
71
Count
72
Count
73
Count
74
Count
75
Count
76
Count
77
Count
78
Count
79
Count
80
Count
81
Count
82
Count
83
Count
84
Count
85
Count
86
Count
87
Count
88
Count
89
Count
90
Count
91
Count
92
Count
93
Count
94
Count
95
Count
96
Count
97
Count
98
Count
99
Count
100
Count
101
Count
102
Count
103
Count
104
Count
105
Count
106
Count
107
Count
108
Count
109
Count
110
Count
11

In [ ]:
data['context'][3]

'In 2006, a toxic waste spill off the coast of Côte d\'Ivoire, from a European ship, prompted the Commission to look into legislation against toxic waste. Environment Commissioner Stavros Dimas stated that "Such highly toxic waste should never have left the European Union". With countries such as Spain not even having a crime against shipping toxic waste, Franco Frattini, the Justice, Freedom and Security Commissioner, proposed with Dimas to create criminal sentences for "ecological crimes". The competence for the Union to do this was contested in 2005 at the Court of Justice resulting in a victory for the Commission. That ruling set a precedent that the Commission, on a supranational basis, may legislate in criminal law – something never done before. So far, the only other proposal has been the draft intellectual property rights directive. Motions were tabled in the European Parliament against that legislation on the basis that criminal law should not be an EU competence, but was reje

In [ ]:
ground_truth[3]

'In which year did a toxic waste spill from a European ship prompt the Commission to look into legislation against waste?'

In [ ]:
lis_questions[3]

['What did the Justice, Freedom and Security Commissioner propose?',
 'What did the Justice, Freedom and Security Commissioner propose to create criminal sentences for?',
 'What did the Justice, Freedom and Security Commissioner propose in 2005?',
 'What did Franco Frattini propose to create criminal sentences for?',
 'What did the Justice, Freedom and Security Commissioner propose to do?',
 'What did Franco Frattini propose?',
 'What did Franco Frattini propose to do?',
 'What did the Justice, Freedom and Security Commissioner propose to do in criminal law?']

In [ ]:
ground_truth=list(data['question'][0:300])
ground_truth_split=[]
def convert(lst):
  return ' '.join(lst).split()
for sentence in ground_truth:
  #print(sentence)
  x=sentence.split()
  ground_truth_split.append(x)


In [ ]:
ground_truth[0]

'How many times did Luther preach in Halle in 1545 and 1546?'

In [ ]:
print(lis_questions)

[['When did Luther preach in Halle?'], ["What is the evidence for Luther's choice of words?"], ['Who is the leader of the party or coalition with the most seats in the Legislative Assembly?'], ['What did the Justice, Freedom and Security Commissioner propose?'], ['What is the largest food-processing industry in Kenya?'], ['What court affirmed that a Virginia pastor had the right to deny local church membership to a man in an openly gay relationship?'], ['What Court of Justice held that CIA Security was entitled to invoke a Directive against her employer?'], ['What can be combined to produce a better view of the subsurface?'], ['What is the name of the new dynasty?'], ['What would have been written Genghis?'], ['What did the Baltimore Ravens kicker Justin Tucker slip and miss?'], ["What did NASA's robotic Lunar Reconnaissance Orbiter photograph?"], ['Who established the East Africa Protectorate?'], ['Who produced the first major work to challenge the bubonic plague theory?'], ['What is 

In [ ]:
#print(lis_questions)
lis_questions_split=[]
for questions in lis_questions:
  lis_q=[]
  for question in questions:
    x=sentence.split()
    lis_q.append(x)
  lis_questions_split.append(lis_q)  


In [ ]:
lis_questions_split

[[['Who',
   'created',
   'an',
   'index',
   'of',
   'health',
   'and',
   'social',
   'problems?'],
  ['Who',
   'created',
   'an',
   'index',
   'of',
   'health',
   'and',
   'social',
   'problems?'],
  ['Who',
   'created',
   'an',
   'index',
   'of',
   'health',
   'and',
   'social',
   'problems?'],
  ['Who',
   'created',
   'an',
   'index',
   'of',
   'health',
   'and',
   'social',
   'problems?'],
  ['Who',
   'created',
   'an',
   'index',
   'of',
   'health',
   'and',
   'social',
   'problems?'],
  ['Who',
   'created',
   'an',
   'index',
   'of',
   'health',
   'and',
   'social',
   'problems?'],
  ['Who',
   'created',
   'an',
   'index',
   'of',
   'health',
   'and',
   'social',
   'problems?'],
  ['Who',
   'created',
   'an',
   'index',
   'of',
   'health',
   'and',
   'social',
   'problems?']],
 [['Who',
   'created',
   'an',
   'index',
   'of',
   'health',
   'and',
   'social',
   'problems?'],
  ['Who',
   'created',
   'an',
   

In [ ]:
from datasets import load_metric
bleu=load_metric("bleu")
predictions=ground_truth_split
references=lis_questions_split
bleu.compute(predictions=predictions,references=references)

{'bleu': 0.005806069503916366,
 'brevity_penalty': 1.0,
 'length_ratio': 1.1577777777777778,
 'precisions': [0.053742802303262956,
  0.0028308563340410475,
  0.0027711797307996833,
  0.0026954177897574125],
 'reference_length': 2700,
 'translation_length': 3126}

In [ ]:
!nlg-eval --setup

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Installing to /root/.cache/nlgeval
In case of incomplete downloads, delete the directory and run `nlg-eval --setup /root/.cache/nlgeval' again.
glove2word2vec.py: 100% 1.00/1.00 [00:00<00:00, 432 chunks/s]


In [ ]:
from nlgeval import NLGEval
nlgeval = NLGEval()  # loads the models
metrics_dict = nlgeval.compute_metrics(ground_truth,lis_questions)

Will try with limiting Meteor to 1GB of memory but this might cause issues.
If you have problems using Meteor, then you can try to lower the `mem` variable in meteor.py
